# 3. MapReduce

For this exercise we are going to use MapReduce in local mode, i.e. we won't be running anything on the cluster!
 
## 3.1. Use the commands `head`, `cat`, `uniq`, `wc`, `sort`, `find`, `xargs`, `awk` to evaluate the NASA log file:

* Data File:  <https://github.com/scalable-infrastructure/exercise-2018/blob/master/data/nasa/NASA_access_log_Jul95.gz>
* Which page was called the most?
* What was the most frequent return code?
* How many errors occurred? What is the percentage of errors?


In [1]:
import os, sys
os.chdir(os.path.join(os.environ["HOME"], "exercise-students-2020/03_MapReduce"))
%pwd

'/home/jupyter-filipiuk-12151613/exercise-students-2020/03_MapReduce'

In [2]:
!head /opt/data/nasa/NASA_access_log_Jul95

199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245
unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985
199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085
burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0
199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0
205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985
d104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985
129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] "GET / HTTP/1.0" 

In [100]:
#Which page was called the most? (Answer: number_of_calls, website)
!awk '{print $1}' /opt/data/nasa/NASA_access_log_Jul95 | sort -n | uniq -c | sort -nr | head -n 1

  17572 piweba3y.prodigy.com
sort: write failed: 'standard output': Broken pipe
sort: write error


In [51]:
#What was the most frequent return code? (Answer: number_of_code, code)
! awk '{print $(NF-1)}' /opt/data/nasa/NASA_access_log_Jul95 | sort -n | uniq -c | sort -nr | head -n 1

1701534 200


In [97]:
#How many errors occurred? What is the percentage of errors?
!x="$(wc -l /opt/data/nasa/NASA_access_log_Jul95 | awk '{print $1}') $(awk '$(NF-1) >= 400 {print $(NF-1)}' /opt/data/nasa/NASA_access_log_Jul95 | wc -l)"; echo $x | awk '{print "Total num of errors: " $2}'; echo $x | awk '{print "Error percentage: "($2 / ($1) * 100) "%"}'

Total num of errors: 10981
Error percentage: 0.580479%


## 3.2 Implement a Python version of this Unix Shell script using this script as template! Run the Python script inside an Hadoop Streaming job.

Template: <https://github.com/scalable-infrastructure/scalable-infrastructure.github.io/blob/master/src/map_reduce.py>

In [56]:
import os
TMP_DIR=os.path.join(os.getcwd(), "tmp")
os.environ["HADOOP_HOME"]="/opt/hadoop-2.8.5"
os.environ["JAVA_HOME"]="/usr"
os.environ["JAVA_OPTS"]="-Djava.io.tmpdir="+TMP_DIR
os.environ["HADOOP_OPTS"]="-Djava.io.tmpdir="+TMP_DIR
os.environ["PATH"]=os.path.join(os.environ["HADOOP_HOME"], "bin") + ":"+os.environ["PATH"]

In [57]:
!hadoop jar ${HADOOP_HOME}/share/hadoop/tools/lib/hadoop-streaming-2.8.5.jar -info

Usage: $HADOOP_PREFIX/bin/hadoop jar hadoop-streaming.jar [options]
Options:
  -input          <path> DFS input file(s) for the Map step.
  -output         <path> DFS output directory for the Reduce step.
  -mapper         <cmd|JavaClassName> Optional. Command to be run as mapper.
  -combiner       <cmd|JavaClassName> Optional. Command to be run as combiner.
  -reducer        <cmd|JavaClassName> Optional. Command to be run as reducer.
  -file           <file> Optional. File/dir to be shipped in the Job jar file.
                  Deprecated. Use generic option "-files" instead.
  -inputformat    <TextInputFormat(default)|SequenceFileAsTextInputFormat|JavaClassName>
                  Optional. The input format class.
  -outputformat   <TextOutputFormat(default)|JavaClassName>
                  Optional. The output format class.
  -partitioner    <JavaClassName>  Optional. The partitioner class.
  -numReduceTasks <num> Optional. Number of reduce tasks.
  -inputreader    <spec> Optional. 

In [86]:
!cat /opt/data/nasa/NASA_access_log_Jul95| python map_reduce.py map | sort | python map_reduce.py reduce

error	10980
ok	1880734


In [109]:
!hadoop jar ${HADOOP_HOME}/share/hadoop/tools/lib/hadoop-streaming-2.8.5.jar -files map_reduce.py -input /opt/data/nasa/ -mapper 'map_reduce.py map' -reducer 'map_reduce.py reduce' -output output_nasa

20/04/14 14:17:29 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
20/04/14 14:17:29 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
20/04/14 14:17:29 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
20/04/14 14:17:29 INFO mapred.FileInputFormat: Total input files to process : 1
20/04/14 14:17:29 INFO mapreduce.JobSubmitter: number of splits:7
20/04/14 14:17:29 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1337391566_0001
20/04/14 14:17:30 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jupyter-filipiuk-12151613/exercise-students-2020/03_MapReduce/map_reduce.py as file:/tmp/hadoop-jupyter-filipiuk-12151613/mapred/local/1586873849917/map_reduce.py
20/04/14 14:17:30 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
20/04/14 14:17:30 INFO mapreduce.Job: Running job: job_local1337391566_0001
2

---

## 3.3 Run the program Terasort on 1 GB of data - each record that TeraGen generates is 100 Bytes in size:

    hadoop jar ${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.5.jar teragen <number_of_records> <output_directory>

    hadoop jar ${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.5.jar terasort <input_directory> <output_directory>
    
    hadoop jar ${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.5.jar teravalidate <input_directory> <output_directory>

Measure the runtime for each step and plot the results! 

In [8]:
!hadoop jar ${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.8.5.jar teragen 

teragen <num rows> <output dir>


In [9]:
!rm -rf teraout-100MB
!hadoop jar ${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.8.5.jar terasort

Usage: terasort [-Dproperty=value] <in> <out>
TeraSort configurations are:
<mapreduce.terasort.num-rows>     Number of rows to generate during teragen.
<mapreduce.terasort.num.partitions>     Number of partitions used for sampling.
<mapreduce.terasort.partitions.sample>     Sample size for each partition.
<mapreduce.terasort.final.sync>     Perform a disk-persisting hsync at end of every file-write.
<mapreduce.terasort.use.terascheduler>     Use TeraScheduler for computing input split distribution.
<mapreduce.terasort.simplepartitioner>     Use SimplePartitioner instead of TotalOrderPartitioner.
<mapreduce.terasort.output.replication>     Replication factor to use for output data files.
